In [24]:
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv("./data/Kickstarter_merged.csv")
ignored_states =  ['canceled', 'live']
df = df[~df.state.isin(ignored_states)]
df.shape

(1828, 26)

In [26]:
df['binary_state'] = np.where(df['state'] == 'successful', 1, 0)
df.columns

Index(['id', 'name', 'description', 'description_story', 'description_risks',
       'rewards', 'category', 'pledged', 'goal', 'deadline', 'location',
       'state', 'faq_count', 'update_count', 'backers_count', 'is_starrable',
       'spotlight', 'staff_pick', 'video', 'creator_name', 'creator_url',
       'url', 'created_at', 'published_at', 'launched_at', 'link',
       'binary_state'],
      dtype='object')

In [27]:
var_cols = [c for c in df.columns if c not in ['binary_state']]
X = df.loc[:, var_cols]
y = df.loc[:, 'binary_state']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1462, 26), (366, 26), (1462,), (366,))

In [29]:
X_train.dtypes

id                     int64
name                  object
description           object
description_story     object
description_risks     object
rewards               object
category              object
pledged              float64
goal                   int64
deadline              object
location              object
state                 object
faq_count             object
update_count         float64
backers_count          int64
is_starrable           int64
spotlight              int64
staff_pick             int64
video                 object
creator_name          object
creator_url           object
url                   object
created_at            object
published_at          object
launched_at           object
link                  object
dtype: object

In [42]:
df['faq_count'].unique()

array(['0', '32', '9', '13', '3', '1', '120', '15', '42', '170', '39',
       '53', '7', '65', '305', '6', '54', '2', '40', '14', '18', '4',
       '10', '67', '5', '17', '25', '72', '58', '970', '20', '101', '44',
       '12', '133', '8', '55', '138', '135', '11', '16', '90', '89',
       '319', '43', '30', '19', '61', '123', '38', '99', '403', '76',
       '47', '46', '34', '29', '745', '71', '178', '141', '127', '163',
       '325', '26', '115', '273', '31', '85', '349', '24', '45', '27',
       '1,203', '41', '129', '49', '231', '48', '576', '400', '239',
       '1,144', '86', '109', '56', '267', '143', '169', '366', '631',
       '145', '303', '218', '328', '59', '1,467', '107', '52', '256',
       '23', '36', '257', '393', '28', '1,449', '399', '367', '114', '78',
       '229', '555', '81', '301', '230', '110', '260', '117', '166',
       '137', '21', '1,109', '155', '124', '51', '22', '159', '500',
       '205', '580', '75', '57', '454', '187', '154', '77', '199', '213',
       

In [28]:
model_xgboost = xgb.XGBClassifier(learning_rate=0.1,
                                      max_depth=5,
                                      n_estimators=5000,
                                      subsample=0.5,
                                      colsample_bytree=0.5,
                                      eval_metric='auc',
                                      verbosity=1)

model_xgboost.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=[(X_test, y_test)],
                  verbose=True)

c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:name, description, description_story, description_risks, rewards, category, deadline, location, state, faq_count, video, creator_name, creator_url, url, created_at, published_at, launched_at, link